## Question 2

#### Implementing random matrix multiplication

For the following problem, I make the following assumptions 

1. $A \in \mathbb{R}^{10X10}$
2. $B \in \mathbb{R}^{10X10}$


In [1]:
# Importing all required libraries
import numpy as np

In [19]:
def randomized_matrix_mul(A,B, C = 10, sampling_type = 'non-uniform'):
    
    n = 10
    # Selecting a suitable r given by r = Clogn 
    # where n is the dimension of the matrix
    
    # tf is going on here figure it out today
    r = int(np.round(C * np.log(n))) 
    
    print(r)
    
    index_range = np.random.choice(range(n), size = r, replace = True)
    print(index_range)
    
    if sampling_type == 'uniform': 
        pk = [1/n] * r
    else:
        total_norm = sum([np.linalg.norm(A[:,k], ord = 2) * np.linalg.norm(B[k,:]) for k in index_range])
        pk = []
    # Generating random indices tha

In [20]:
randomized_matrix_mul(1,1)

23
[5 4 7 2 3 5 3 3 6 6 1 0 6 4 4 5 2 1 4 0 2 1 0]


TypeError: 'int' object is not subscriptable